In [61]:
#!/usr/bin/env python
'''
Pymodbus Synchrnonous Client Test with Dynasonic DXN Energy Meter
--------------------------------------------------------------------------

The following is an example of how to use the synchronous modbus client
implementation from pymodbus. This has been adapted from a sample script 
at https://pythonhosted.org/pymodbus/examples/synchronous-client.html

_Additional Note from sample script:
It should be noted that the client can also be used with
the guard construct that is available in python 2.5 and up::

    with ModbusClient('127.0.0.1') as client:
        result = client.read_coils(1,10)
        print result
        
           
***Created 2018-07-22 by Chris Weyandt        
'''

#---------------------------------------------------------------------------# 
# import the required server implementation
#---------------------------------------------------------------------------# 
from pymodbus.client.sync import ModbusTcpClient as ModbusClient
#from pymodbus.client.sync import ModbusUdpClient as ModbusClient
#from pymodbus.client.sync import ModbusSerialClient as ModbusClient

#additional imports for conversions
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadDecoder

#---------------------------------------------------------------------------# 
# configure the client logging
#---------------------------------------------------------------------------# 
import logging
logging.basicConfig()
log = logging.getLogger()
log.setLevel(logging.DEBUG)

In [68]:
#---------------------------------------------------------------------------# 
# Dynasonic DXN Energy Meter Modbus Register List
#---------------------------------------------------------------------------# 
# https://www.badgermeter.com/resources/2364ad87-a49a-42ec-9bd7-98f5e7c1689f/dynasonics%20dxn%20portable%20clamp-on%20ultrasonic%20flow%20meter%20user%20manual%20hyb-um-00090-en.pdf/
#
# ['Signal Name', modbus_offset, num_bytes, 'unit']
#
# must be in inclusive sequential order (i.e. must define every set of 
# registers in a given block)
# In current implementation, the modbus_offset and num_bytes parameters are 
# ignored - the decoder just processes the list in order as it pulls bytes
# off of the read buffer.
#---------------------------------------------------------------------------# 
reg_list = []
reg_list.append( ['Signal Strength', 0, 2, '%'] )
reg_list.append( ['Flow Rate', 2, 2, 'gpm'] )
reg_list.append( ['Net Total', 4, 2, 'gal'] )
reg_list.append( ['Positive Totalizer', 6, 2, 'gal'] )
reg_list.append( ['Negative Totalizer', 8, 2, 'gal'] )
reg_list.append( ['Temp1', 10, 2, 'degF'] )
reg_list.append( ['Temp2', 12, 2, 'degF'] )

In [69]:
#---------------------------------------------------------------------------# 
# choose the client you want
#---------------------------------------------------------------------------# 
# make sure to start an implementation to hit against. For this
# you can use an existing device, the reference implementation in the tools
# directory, or start a pymodbus server.
#---------------------------------------------------------------------------# 

IP_ADDRESS = '128.3.5.24'

client = ModbusClient( IP_ADDRESS )

In [71]:
#---------------------------------------------------------------------------# 
# read register range from device
#---------------------------------------------------------------------------# 
# format is 
# read_holding_registers(start_register-1, num_registers, unit=device_num)
# I'm unsure why you have to subtract 1 from the starting register, but you do
#
#
# Expected Results
#
# SUCCESS: 
#     type(rr) = pymodbus.register_read_message.ReadHoldingRegistersResponse
#     print(rr) = ReadRegisterResponse (14)
#
# Thrown when unable to connect to IP address:
#     ConnectionException: Modbus Error: [Connection] 
#         Failed to connect[ModbusTcpClient(128.3.5.22:502)]
#     type(rr) = invalid, as variable assignment was not executed successfully
#
# INVALID REGISTER: 
#     type(rr) = pymodbus.pdu.ExceptionResponse
#     print(rr) = Exception Response(131, 3, IllegalAddress)
#---------------------------------------------------------------------------# 

START_REGISTER = 200
NUM_REGISTERS = 14
UNIT_ID = 1

rr = client.read_holding_registers(START_REGISTER-1, NUM_REGISTERS, unit=UNIT_ID)

print(type(rr), '\n', rr)

DEBUG:pymodbus.transaction:Current transaction state - TRANSCATION_COMPLETE
DEBUG:pymodbus.transaction:Running transaction 2
DEBUG:pymodbus.transaction:SEND: 0x0 0x2 0x0 0x0 0x0 0x6 0x1 0x3 0x0 0xc7 0x0 0xe
DEBUG:pymodbus.client.sync:New Transaction state 'SENDING'
DEBUG:pymodbus.transaction:Changing transaction state from 'SENDING' to 'WAITING FOR REPLY'
DEBUG:pymodbus.transaction:Changing transaction state from 'WAITING FOR REPLY' to 'PROCESSING REPLY'
DEBUG:pymodbus.transaction:RECV: 0x0 0x2 0x0 0x0 0x0 0x1f 0x1 0x3 0x1c 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x21 0xcb 0x42 0x9d 0x6c 0x8b 0x42 0xa2
DEBUG:pymodbus.framer.socket_framer:Processing: 0x0 0x2 0x0 0x0 0x0 0x1f 0x1 0x3 0x1c 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x21 0xcb 0x42 0x9d 0x6c 0x8b 0x42 0xa2
DEBUG:pymodbus.factory:Factory Response[ReadHoldingRegistersResponse: 3]
DEBUG:pymodbus.transaction:Adding transaction 2
DEBUG:pymodbus.transact

<class 'pymodbus.register_read_message.ReadHoldingRegistersResponse'> 
 ReadRegisterResponse (14)


In [72]:
#---------------------------------------------------------------------------# 
# Decode payload into proper format
# Each type that decode_32bit_float runs, it pulls four bytes off the buffer
# so it will fail if the number of read bytes is odd, or if you try to read
# past the end of the buffer.
#---------------------------------------------------------------------------# 

dec = BinaryPayloadDecoder.fromRegisters(
    rr.registers,
    byteorder=Endian.Big,
    wordorder=Endian.Little)
for metric in reg_list:
    print(metric[0] + ":  " + str(dec.decode_32bit_float()) + " " + metric[3])

        
#---------------------------------------------------------------------------# 
# close the client
#---------------------------------------------------------------------------# 
client.close()

DEBUG:pymodbus.payload:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8651, 17053, 27787, 17058]
DEBUG:pymodbus.payload:b'\x00\x00\x00\x00'
DEBUG:pymodbus.payload:b'\x00\x00\x00\x00'
DEBUG:pymodbus.payload:b'\x00\x00\x00\x00'
DEBUG:pymodbus.payload:b'\x00\x00\x00\x00'
DEBUG:pymodbus.payload:b'\x00\x00\x00\x00'
DEBUG:pymodbus.payload:b'B\x9d!\xcb'
DEBUG:pymodbus.payload:b'B\xa2l\x8b'


Signal Strength:  0.0 %
Flow Rate:  0.0 gpm
Net Total:  0.0 gal
Positive Totalizer:  0.0 gal
Negative Totalizer:  0.0 gal
Temp1:  78.56600189208984 degF
Temp2:  81.21199798583984 degF


In [73]:
?BinaryPayloadDecoder

Init signature: BinaryPayloadDecoder(payload, byteorder='<', wordorder='>')
Docstring:     
A utility that helps decode payload messages from a modbus
reponse message.  It really is just a simple wrapper around
the struct module, however it saves time looking up the format
strings. What follows is a simple example::

    decoder = BinaryPayloadDecoder(payload)
    first   = decoder.decode_8bit_uint()
    second  = decoder.decode_16bit_uint()
Init docstring:
Initialize a new payload decoder

:param payload: The payload to decode with
:param byteorder: The endianess of the payload
:param wordorder: The endianess of the word (when wordcount is >= 2)
File:           ~/anaconda3/lib/python3.6/site-packages/pymodbus/payload.py
Type:           type
